# Using vireo for clonal reconstruction - mitochondrial mutations

The mitochondrial mutations data set is extracted from [Ludwig et al, Cell, 2019](https://doi.org/10.1016/j.cell.2019.01.022), the 9 variants used here are from Supp Fig. 2F (and main Fig. 2F).

For clonal reconstruction, we are using the core class `vireoSNP.Vireo` with multiple initializations

In [1]:
import vireoSNP
import numpy as np
from scipy import sparse
from scipy.io import mmread
import matplotlib.pyplot as plt

print(vireoSNP.__version__)

0.4.2


In [2]:
import os
import pandas as pd
from src.config import RESULTS, FIGURES_DIR, ROOT_DIR
from src.utils.parse_config import read_config_file
os.chdir(ROOT_DIR)



Project Directory: /data2/mito_lineage


In [3]:
data_df = pd.read_csv("parameters/2020_11_18_Croker_mito.csv", index_col=0)
data_df

,raw,barcode_f,bam_f
sample,,,
PBMC_J,NaN,/data2/isshamie/mito_lineage/data/processed/mt...,/data2/isshamie/mito_lineage/data/processed/mt...
PBMC_P,NaN,/data2/isshamie/mito_lineage/data/processed/mt...,/data2/isshamie/mito_lineage/data/processed/mt...


In [4]:
PBMC_J = os.path.join(os.path.dirname(data_df.loc["PBMC_J", "bam_f"]), "filtered_peak_bc_matrix/matrix.mtx")

In [5]:
PBMC_P = os.path.join(os.path.dirname(data_df.loc["PBMC_P", "bam_f"]), "filtered_peak_bc_matrix/matrix.mtx")

In [6]:
np.set_printoptions(formatter={'float': lambda x: format(x, '.5f')})

In [7]:
AD = mmread(PBMC_J).tocsc()

DP = mmread(PBMC_P).tocsc()
AD

<65851x743 sparse matrix of type '<class 'numpy.int64'>'
	with 2076834 stored elements in Compressed Sparse Column format>

In [8]:
# AD = mmread("../data/mitoDNA/cellSNP.tag.AD.mtx").tocsc()
# DP = mmread("../data/mitoDNA/cellSNP.tag.DP.mtx").tocsc()

## New implementation in v0.3.0

In [9]:
from vireoSNP import Vireo

Note, there are a few key parameters:

* `n_donor`: it means number of clones here, but you need to determine yourself. You can choose the one with highest ELBO (see next section)
* `n_GT`: number of different genotype. For somatic mutations, we could assume that there are only two, namely present or absent
* `is_theta_ASE`: whether allelic ratio is variant specific. For mitochondrial mutations, we choose True as allelic ratio can different from variant to variant

### Multiple initialization is needed to avoid local optima

In [11]:
np.random.seed(1)

n_initials = 50
_models_all = []
for i in range(n_initials):
    _modelCA = Vireo(n_var=9, n_cell=81, n_donor=3, n_GT=2,
                     fix_beta_sum=False, ASE_mode=True)
    _modelCA.set_prior(beta_mu_prior=np.array([[0.01, 0.5]]))
    _modelCA.fit(AD, DP, min_iter=20, verbose=False, delay_fit_theta=0)
    _models_all.append(_modelCA)

ValueError: dimension mismatch

In [ ]:
## Choose the model giving highest ELBO

_idx = np.argmax([x.ELBO_[-1] for x in _models_all])
modelCA = _models_all[_idx]
_losses = modelCA.ELBO_

In [ ]:
fig = plt.figure(figsize=(11, 4))
plt.subplot(1, 2, 1)
plt.hist([x.ELBO_[-1] for x in _models_all])
plt.ylabel("Frequency")
plt.xlabel("ELBO with multiole initializations")

plt.subplot(1, 2, 2)
plt.plot(_losses)
plt.xlabel("Iterations")
plt.ylabel("ELBO in a single run")

plt.tight_layout()
plt.show()

In [ ]:
print(modelCA.beta_mu),
print(modelCA.beta_sum)

In [ ]:
import matplotlib.pyplot as plt
from vireoSNP.plot.base_plot import heat_matrix

fig = plt.figure(figsize=(7, 4), dpi=100)
plt.subplot(1, 2, 1)
im = heat_matrix(modelCA.ID_prob, cmap="Oranges", alpha=0.8,
                 display_value=False, row_sort=True)
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title("Assignment probability")
plt.xlabel("Clone")
plt.ylabel("%d cells" %(modelCA.n_cell))
plt.xticks(range(modelCA.n_donor))


plt.subplot(1, 2, 2)
AF_SNPs = np.sum(modelCA.GT_prob * 
                 np.expand_dims(modelCA.beta_mu, 1), axis=2)
im = heat_matrix(AF_SNPs, cmap="Blues", alpha=0.8, 
                 display_value=False, row_sort=True)
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title("Mean allelic ratio")
plt.xlabel("Clone")
plt.ylabel("%d SNPs" %(modelCA.n_var))
plt.xticks(range(modelCA.n_donor))

plt.tight_layout()
plt.show()

In [ ]:
np.sum(modelCA.predict_doublet(AD, DP)[0])

### Choose the number of clones
We can choose the number of clones where ELBO stops increasing. Here, the best number of clone is 3.

Note, for a certain number of clone, e.g., n_clone=3, the ELBO may not the same for multiple initializations. This is becaused VI can only guarantee a local optima, so please choose the highest ELBO for each `n_clone` to represent its "global" optima.

In [ ]:
n_initials = 50
n_clone_list = np.arange(2, 6)

_ELBO_mat = [] 
for k in n_clone_list:
    _elbo_temp = []
    for i in range(50):
        _modelCA = Vireo(n_var=9, n_cell=81, n_donor=k, n_GT=2,
                         fix_beta_sum=False, ASE_mode=True)
        _modelCA.set_prior(beta_mu_prior=np.array([[0.01, 0.5]]))
        _modelCA.fit(AD, DP, min_iter=20, verbose=False)
        _elbo_temp.append(_modelCA.ELBO_[-1])
    _ELBO_mat.append(_elbo_temp)

In [ ]:
plt.plot([1, 2, 3, 4], np.max(_ELBO_mat, axis=1))
plt.boxplot(_ELBO_mat)
plt.xticks(n_clone_list - 1, n_clone_list)
plt.ylabel("ELBO")
plt.xlabel("n_clones")
plt.show()

### Old version

The old version seem more likely to end up at local optima in this data set.

In [ ]:
from vireoSNP.utils.vireo_wrap_v02 import vireo_flock

In [ ]:
# theta_prior = np.array([[0.01, 10], [0.4, 10], [5, 5]])
theta_prior = np.array([[0.1, 10], [5, 5]])
res = vireo_flock(AD, DP, n_donor=3, learn_GT=True,
                  n_extra_donor=0, ASE_mode=True,
                  theta_prior=theta_prior, learn_theta=True,
                  n_init=50, check_doublet=False, random_seed=1)

In [ ]:
plt.plot(res['LB_list'])
plt.xlabel("Iterations")
plt.ylabel("ELBO in a single run")
plt.show()

In [ ]:
## If ASE_mode is True
AF_mean = res['theta_shapes'][:, 0] / np.sum(res['theta_shapes'], axis=1)

_new_shape = (AF_mean.shape[1], 1, AF_mean.shape[0])
AF_SNPs = np.sum(res['GT_prob'] * AF_mean.transpose().reshape(_new_shape), axis=2)

In [ ]:
import matplotlib.pyplot as plt
from vireoSNP.plot.base_plot import heat_matrix

fig = plt.figure(figsize=(7, 4), dpi=100)
plt.subplot(1, 2, 1)
im = heat_matrix(res['ID_prob'], cmap="Oranges", alpha=0.8,
                 display_value=False, row_sort=True)
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title("Assignment probability")
plt.ylabel("%d cells" %(res['ID_prob'].shape[0]))
plt.xlabel("Clone")
plt.xticks([0, 1, 2])

plt.subplot(1, 2, 2)
im = heat_matrix(AF_SNPs, cmap="Blues", alpha=0.8, 
                 display_value=False, row_sort=True)
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title("Mean allelic ratio")
plt.xlabel("Clone")
plt.ylabel("%d SNPs" %(AF_SNPs.shape[0]))
plt.xticks([0, 1, 2])
plt.xlim(-0.5, 2.5)

plt.tight_layout()
plt.show()